# SORA - Stellar Occultation Reduction and Analysis

## Observer Object Class

The Observer Object Class within SORA was created to deal with the observer location. The documentation at <font color=blue>add link</font> contains the details about every step.  

This Jupyter-Notebook was designed as a tutorial for how to work with the Observer Object Class. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.

### Index

1. [Instantiating the Observer Object](#section_1)
2. [Setting and/or modifing parameters](#section_2)
3. [Apparent Sidereal Time](#section_3)
4. [Ksi and Eta projection](#section_4)
5. [MPC observatories](#section_5)

In [2]:
## import the Observer Class
from sora.observer import Observer
## This means the Observer Class is imported from the "observer" module of the sora package

## To facilitate, sora allows to import Observer directly from the sora package.
from sora import Observer

<a id="section_1"></a>
### 1. Instantiating the Observer Object

The Observer Object Class can be instantiated in different ways. First, all Observers should have a 'name' that will distinguish it from other Observer Objects and also allows the Occultation Object to control all the steps and Objects. This is the name which the user will use to refer to this observer within the Occultation Object. It is not possible to define two different Observer Objects with the same name. The name is not needed when the site is downloaded from the MPC database in which case the name in the MPC will be used.

The Observer main objective is to deal with the observer location, convert the coordinates from ITRS and GCRS and vice-versa and calculate the Orthographic projection the coordinates in the direction of a coordinate.

**The Observer Docstring was designed to help the users. Also, each function has its Docstring containing its main purpose and the needed parameters (physical description and formats). Please, do not hesitate to use it.**

In [3]:
Observer?

**Example of Observer intantiated with a MPC code**

In [4]:
opd = Observer(code='874')

In [5]:
## To see the name of the object "opd"
opd.name

'Observatorio do Pico dos Dias, Itajuba'

In this case, the name is a bit too long, the user can define a differente name upon instantiation.

In [6]:
opd = Observer(name='OPD', code='874')

**Example of Observer instantiated with coordinates**

This coordinates must in degrees for longitude and latitude and in meters in height.

In [7]:
# using string representation
obs1 = Observer(name='Ex 1', lon='-45 34 57', lat='-22 32 04', height=1864)

In [8]:
# using numeric representation
obs2 = Observer(name='Ex 2', lon=-69.295805, lat=-31.79877, height=2495)

**Example using an Astropy EarthLocation Object**

Astropy has an EarthLocation Class with observer location information. SORA uses this class as a core functionality in the Observer Class. So the user can give an object create from EarthLocation as input.

In [9]:
from astropy.coordinates import EarthLocation
site = EarthLocation('-45 34 57', ' -22 32 04', 1864)
obs3 = Observer(name='Ex 3', site=site)

**An error will rise if the user triesto intanciate two LightCurves with the same name**

In [10]:
obs4 = Observer(name='Ex 4', lon='-45 34 57', lat='-22 32 04', height=1864)

obs5 = Observer(name='Ex 4', lon='+45 34 57', lat='+22 32 04', height=300)

ValueError: name Ex 4 already defined for another Observer object. Please choose a different one.

If the user wish to overwrite the object with given name, the user must delete the previous object first.

In [11]:
del(obs4)

obs5 = Observer(name='Ex 4', lon='+45 34 57', lat='+22 32 04', height=300)

**The user can access some information from the object as attribute**

In [12]:
obs5.name

'Ex 4'

In [13]:
obs5.lon

<Longitude 45.5825 deg>

In [14]:
obs5.lat

<Latitude 22.53444444 deg>

In [15]:
obs5.height

<Quantity 300. m>

If the user wants to see the informations in the object, just prints the object directly.

In [16]:
print(obs5)

Site: Ex 4
Geodetic coordinates: Lon: 45d34m57s, Lat: 22d32m04s, height: 0.300 km


<a id="section_2"></a>
### 2. Setting and/or modifing parameters

If any parameter given to Observer is not correct, the user does not need to delete or overwrite the previous object. It can be given the new values directly to the object attributes. Only the "name" attribute cannot be replaced.

In [17]:
print(obs5.lon)

45d34m57s


In [18]:
obs5.lon = '-35 40 26'
print(obs5.lon)

-35d40m26s


In [19]:
obs5.lon = 67.3514
print(obs5.lon)

67d21m05.04s


In [20]:
obs5.name = 'Teste'

AttributeError: can't set attribute

<a id="section_3"></a>
### 3. Apparent Sidereal Time

If the user, for some reason, wants to know the apparent sidereal time at a certain location for a given time, there exists a function that calculates it.

In [21]:
obs5.sidereal_time?

In [22]:
obs5.sidereal_time('2020-05-10 00:00:00')

<Longitude 19.70677566 hourangle>

The input time can also be an Astropy Time Object

In [23]:
from astropy.time import Time
t = Time('2020-05-10 00:00:00')

obs5.sidereal_time(t)

<Longitude 19.70677566 hourangle>

If the user wants to calculate the Greenwich Apparent Sidereal Time, it can be done as:

In [24]:
obs5.sidereal_time('2020-05-10 00:00:00', mode='greenwich')

<Longitude 15.21668232 hourangle>

<a id="section_4"></a>
### 4. Ksi and Eta projection

Observer object calculates the orthographic projection (ksi and eta) of a site in the direction of a star given the following function. Ksi is in the East direction and Eta in the North direction.

The coordinates of the star given must be in the Geocentric Celestial Reference System (GCRS).

This is calculated automaticaly in Occultation.

In [25]:
obs5.get_ksi_eta?

In [26]:
ksi, eta = obs5.get_ksi_eta(time='2020-05-10 00:00:00', star='19 21 18.63201 -21 44 25.3924')
print(ksi, eta)

519.9775286976982 4427.425747705024


A list of times can be given to the time param, as shown below.

In [27]:
times = ['2020-05-10 00:00:00', '2020-05-10 01:00:00', '2020-05-10 02:00:00', '2020-05-10 03:00:00']
ksi, eta = obs5.get_ksi_eta(time=times, star='19 21 18.63201 -21 44 25.3924')
print(ksi)
print(eta)

[ 519.9775287  2026.0979061  3393.69312852 4529.05575985]
[4427.42574771 4301.02229756 4034.56965075 3646.32512117]


<a id="section_5"></a>
### 5. MPC observatories

The user can download and look all the observatories from the MPC database. These are ground-based observatories. The spacecraft observatories are ignored. The function presented returns a Python dictionary that can be searched by the MPC code.

In [28]:
from sora.observer import search_code_mpc
observatories = search_code_mpc()

In [29]:
observatories['874']

('Observatorio do Pico dos Dias, Itajuba',
 <EarthLocation (4126.27235742, -4211.05894345, -2429.98090308) km>)

In [30]:
observatories['511']

('Haute Provence',
 <EarthLocation (4578.31238503, 458.24339505, 4403.08309658) km>)

In [31]:
observatories

{'000': ('Greenwich', <EarthLocation (3980.65908307, 0., 4966.84662601) km>),
 '001': ('Crowborough',
  <EarthLocation (4017.70150872, 10.81285696, 4937.37963307) km>),
 '002': ('Rayleigh',
  <EarthLocation (3966.96894668, 42.92841063, 4981.324997) km>),
 '003': ('Montpellier',
  <EarthLocation (4613.44108454, 314.51286042, 4381.780119) km>),
 '004': ('Toulouse',
  <EarthLocation (4623.91819079, 118.053188, 4377.12407899) km>),
 '005': ('Meudon',
  <EarthLocation (4205.68487713, 163.84501614, 4776.42734587) km>),
 '006': ('Fabra Observatory, Barcelona',
  <EarthLocation (4786.95713426, 177.55185536, 4197.63692567) km>),
 '007': ('Paris',
  <EarthLocation (4202.69234465, 171.49762241, 4778.64693755) km>),
 '008': ('Algiers-Bouzareah',
  <EarthLocation (5106.30535438, 270.78274243, 3799.96646186) km>),
 '009': ('Berne-Uecht',
  <EarthLocation (4324.63496703, 564.87289002, 4638.1812264) km>),
 '010': ('Caussols',
  <EarthLocation (4581.93585771, 556.19867516, 4389.0193045) km>),
 '011': (

**This Jupyter-Notebook was designed as a tutorial for how to work with the Observer Object Class. More information about the other classes, please refer to their specif Jupyter-Notebook. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.**

# The End